## Run pipeline "pipeline_route" on Azure

### Imports

In [ ]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

from azureml.pipeline.core import PublishedPipeline
print("Pipeline SDK-specific imports completed")


# REST imports
from azureml.core.authentication import InteractiveLoginAuthentication
import requests
import json
print('REST specific imports completed')


### Settings

In [ ]:
published_pipeline_name = 'Route_Data_Pipeline'
experiment_name = 'pipeline_route_REST'

### Parameters

In [ ]:
param_input_data = '.'    #<=====   Input required base data folder here (default ./data)
param_ml_service = 'TWV'       # <=====   Input required ml service here (eg PAD, TWV..)

### Load workspace, datastore

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config(path='azure_config_dev.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')


## Get published pipeline

In [ ]:
all_published_pipelines = PublishedPipeline.get_all(ws)
published_pipeline = None
for pipeline in all_published_pipelines:
    if  pipeline.name == published_pipeline_name and pipeline.status == 'Active':
        published_pipeline = pipeline
        break

if published_pipeline is None:
    print('Error - no active published pipeline found with name: ',published_pipeline_name)
else:    
    display(published_pipeline)

## Run using REST

In [ ]:
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint = published_pipeline.endpoint

print("Performing HTTPS POST on URL {} to trigger this pipeline".format(rest_endpoint))

response = requests.post(rest_endpoint, 
                         headers=aad_token, 
                         json={"ExperimentName": experiment_name,
                               "RunSource": "SDK",
                               "ParameterAssignments": {"input_data": param_input_data,
                                                        "ml_service": param_ml_service
                                                       }})


run_id = response.json()["Id"]

print(run_id)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()
aad_token

In [ ]:
response.json()